In [1]:
import tushare as ts

In [2]:
ts.set_token('3463f0f3c5113c101cd9831597c579b4e429e5751f3f1739c6af830a')

In [3]:
pro = ts.pro_api()

600519.SH

贵州茅台


In [15]:
df1 = pro.daily(ts_code='600519.SH', end_date='20250722')

In [16]:
df1

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,600519.SH,20250722,1444.00,1466.60,1440.68,1464.98,1443.00,21.98,1.5232,42118.98,6127636.851
1,600519.SH,20250721,1436.99,1446.80,1435.00,1443.00,1437.00,6.00,0.4175,26203.31,3779891.829
2,600519.SH,20250718,1420.00,1437.81,1416.38,1437.00,1416.35,20.65,1.4580,41823.41,5984949.774
3,600519.SH,20250717,1413.98,1423.01,1412.88,1416.35,1411.03,5.32,0.3770,23105.54,3273100.406
4,600519.SH,20250716,1410.01,1417.49,1409.95,1411.03,1411.00,0.03,0.0021,21286.05,3006882.579
...,...,...,...,...,...,...,...,...,...,...,...
5719,600519.SH,20010831,37.15,37.62,36.80,37.01,37.10,-0.09,-0.2400,23231.48,86231.237
5720,600519.SH,20010830,36.28,37.51,36.00,37.10,36.38,0.72,1.9800,48013.06,177558.558
5721,600519.SH,20010829,36.98,37.00,36.10,36.38,36.86,-0.48,-1.3000,53252.75,194689.620
5722,600519.SH,20010828,34.99,37.00,34.61,36.86,35.55,1.31,3.6900,129647.79,463463.143


In [18]:
type(df1)

pandas.core.frame.DataFrame

In [20]:
df1.to_csv('600519.SH.csv', index=False)

In [21]:
ndsd = pro.daily(ts_code='300750.SZ', end_date='20250722')

In [22]:
ndsd

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,300750.SZ,20250722,280.00,290.00,279.18,286.66,279.66,7.00,2.5030,333090.29,9468912.161
1,300750.SZ,20250721,277.00,282.00,273.55,279.66,271.20,8.46,3.1195,324862.75,9047295.149
2,300750.SZ,20250718,268.70,272.45,265.80,271.20,265.50,5.70,2.1469,221865.58,5981601.235
3,300750.SZ,20250717,265.37,267.75,264.36,265.50,264.40,1.10,0.4160,145110.17,3852746.251
4,300750.SZ,20250716,268.50,268.70,263.04,264.40,267.75,-3.35,-1.2512,156264.79,4135841.242
...,...,...,...,...,...,...,...,...,...,...,...
1722,300750.SZ,20180615,53.00,53.00,53.00,53.00,48.18,4.82,10.0000,2565.10,13595.030
1723,300750.SZ,20180614,48.18,48.18,48.18,48.18,43.80,4.38,10.0000,742.67,3578.184
1724,300750.SZ,20180613,43.80,43.80,43.80,43.80,39.82,3.98,10.0000,450.30,1972.314
1725,300750.SZ,20180612,39.82,39.82,39.82,39.82,36.20,3.62,10.0000,265.79,1058.375


In [23]:
ndsd.to_csv('ndsd_300750.SZ.csv', index=False)

In [24]:
joyson = pro.daily(ts_code='600699.SH', end_date='20250722')
joyson.to_csv('joyson_600699.SH.csv', index=False)

In [25]:
tpjt = pro.daily(ts_code='601689.SH', end_date='20250722')
tpjt.to_csv('tpjt_601689.SH.csv', index=False)

In [ ]:
pinganbank = pro.daily(ts_code='000001.SZ', end_date='20250722')
pinganbank.to_csv('p_000001.SZ.csv', index=False)

In [29]:
xm

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
